In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pprint # For pretty printing the output list

In [5]:
test_round = 2
# Construct the URL for the specified round
round_num = str(test_round)
url = f"https://keeptradecut.com/trade-calculator?var=5&pickVal=0&teamOne=2025{round_num}1|2025{round_num}2|2025{round_num}3|2025{round_num}4|2025{round_num}5|2025{round_num}6|2025{round_num}7|2025{round_num}8|2025{round_num}9|2025{round_num}10|2025{round_num}11|2025{round_num}12&teamTwo=&format=1&isStartup=0"

print(f"--- Testing Round {round_num} ---")
print(f"Fetching URL: {url}")

# Set headers to mimic a browser (often helps avoid blocks)
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'
}

extracted_picks_data = [] # List to store results

--- Testing Round 2 ---
Fetching URL: https://keeptradecut.com/trade-calculator?var=5&pickVal=0&teamOne=202521|202522|202523|202524|202525|202526|202527|202528|202529|2025210|2025211|2025212&teamTwo=&format=1&isStartup=0


In [9]:


try:
    # Make the HTTP request
    print("Sending request...")
    response = requests.get(url, headers=headers, timeout=30) # 30-second timeout
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    print(f"Request successful (Status Code: {response.status_code})")

    # Parse the HTML content
    print("Parsing HTML with BeautifulSoup...")
    soup = BeautifulSoup(response.content, features="html.parser")
    
    print(soup.prettify())  # Print the entire HTML for debugging

    # Find all the relevant divs containing pick info
    # Using the same class as in your original Selenium code
    pick_wrappers = soup.findAll('div', 'team-player-wrapper')
    print(f"Found {len(pick_wrappers)} 'team-player-wrapper' divs.")

    if not pick_wrappers:
        print("\nWARNING: No pick wrappers found. The data might require JavaScript OR the page structure changed.")
    else:
        print("\nExtracting data from wrappers:")
        # Loop through each found div and extract the details
        for wrapper in pick_wrappers:
            draft_position_tag = wrapper.find('p', class_='player-name')
            draft_value_tag = wrapper.find('div', class_='player-value')

            # Check if the main tags were found
            if draft_position_tag and draft_value_tag:
                draft_position_span = draft_position_tag.find('span')
                draft_value_p = draft_value_tag.find('p')

                # Check if the inner tags were found
                if draft_position_span and draft_value_p:
                    # Extract text
                    draft_position = draft_position_span.get_text(strip=True) # e.g., "2025 1.01"
                    draft_pos_value = draft_value_p.get_text(strip=True)      # e.g., "7500"

                    # Parse the draft position string (with error checking)
                    dpos = draft_position.split(' ')
                    pick_year = dpos[0] if len(dpos) > 0 else "YYYY"
                    pick_details = dpos[-1] if len(dpos) > 0 else "R.PP"

                    if '.' in pick_details:
                        pick_round_str, pick_number_str = pick_details.split('.', 1)
                        pick_round = pick_round_str if pick_round_str.isdigit() else 'R'
                        pick_number = pick_number_str if pick_number_str.isdigit() else 'PP'
                    else:
                        pick_round = 'R'
                        pick_number = 'PP'

                    draf_pos_formatted = f"{pick_year} Round {pick_round} Pick {pick_number}"
                    # Generate the KTC ID (same logic as before)
                    ktc_player_id = pick_details.replace('.', '') + '000'
                    # Get current time
                    enrty_time = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f%z")

                    # Append extracted data as a dictionary (easier to read)
                    extracted_picks_data.append({
                        "draft_position": draf_pos_formatted,
                        "ktc_player_id": ktc_player_id,
                        "value": draft_pos_value,
                        "entry_time": enrty_time
                    })
                else:
                    print("  - WARNING: Could not find inner span/p tags in a wrapper.")
            else:
                print("  - WARNING: Could not find outer player-name/player-value tags in a wrapper.")

except requests.exceptions.RequestException as e:
    print(f"\n--- HTTP Request Failed ---")
    print(f"Error: {e}")
except Exception as e:
    print(f"\n--- An Error Occurred ---")
    print(f"Error: {e}")

Sending request...
Request successful (Status Code: 200)
Parsing HTML with BeautifulSoup...
<!DOCTYPE html>
<html>
 <head>
  <script>
   var gtag = false;
  </script>
  <!-- Global site tag (gtag.js) - Google Analytics -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-134209803-1">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
            function gtag() { dataLayer.push(arguments); }
            gtag('js', new Date());

            gtag('config', 'UA-134209803-1');
  </script>
  <script data-canonical="false" data-site="LPLQWNPN" defer="" src="https://cdn.usefathom.com/script.js">
  </script>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta charset="utf-8"/>
  <meta content="utf-8" http-equiv="encoding"/>
  <meta content="notranslate" name="google">
   <meta content="en_US" http-equiv="Content-Language">
    <title>
     Dynasty Football Trade Calculator - KeepTradeCut
    </title>
    <link href="https:/

/var/folders/80/szd4zkw52_54wprklrxqq0280000gn/T/ipykernel_42832/3562478017.py:16: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  pick_wrappers = soup.findAll('div', 'team-player-wrapper')


In [7]:
# Print the final extracted data
print("-" * 40)
print(f"Total picks extracted for Round {round_num}: {len(extracted_picks_data)}")
print("Extracted Data:")
pprint.pprint(extracted_picks_data)
print("-" * 40)

----------------------------------------
Total picks extracted for Round 2: 0
Extracted Data:
[]
----------------------------------------
